# Profiling

## Why profile?

Profiling is the act of finding the demands placed on your computer when a piece of code is run and, preferably, localising the demands to specific sections of code. This is important when optimising as it allows you to find which parts of your code contribute to the demand you want to reduce and then target your optimisation efforts there.

For instance, if ```function_1``` takes 99% of the runtime of your code and ```function2``` takes the rest, there is no point in optimising ```function2``` as even reducing the runtime to zero for that function would not produce a noticable impact on the runtime of the code as a whole. Instead, optimising ```function1``` should be the priority.

For this reason, when optimising a code, the first step is to profile it. This means the time you spend optimising your code will have most impact.

## The Effects of Profiling

One important note when using any profiling technique is that it can impact the run-time and memory usage of the code. This is because tracking various aspects of performance can actually impact performance as more information needs to be collected. As such, it's sometimes necessary to run a profiler on a reduced scale problem so the code runs in a reasonable time.

## Profiling Tools

There are a number of different profiling tools available for use in Python. In this notebook, we will look at a sample of them - favouring ones which are simple to implement in a Jupyter Notebook. However, this list is far from exhaustive.

The examples in this section relate to running this notebook in Colab. to run this notebook in Anaconda locally a different approach may be required in some cases. Many of the commands are also available from the command line when profiling a ```.py``` file. Again, a different approach will be required in this usage case.

## Profiling Effectively

### Profiling Runs

In many cases, when profiling and optimising a large and complex code, it is not possible to perform full-scale executions of the code. This may be because, for example, the code is designed to run on HPC resources but you're profiling on your desktop. Or it may be that you expect the codes to take hours or days to run and you want to make your profiling and optimising cycle shorter.

In addition, it may be that a code may be used in a number of different ways, such as performing many different kinds of simulation or analysing many different kinds of data set. It may be that the code can be run with a multitude of different options that will invoke different parts of the code.

These considerations may mean that simply "profiling the code" is not a striaghtforward affair. Which use cases do you profile? It may be that you need to develop a suite of cases to profile to get a measure of how the code performs in different scenarios.

### Length of Profiling Runs

One other consideration that should always be taken into account is that a very short piece of code will be prone to relatively larger noise than a longer execution of code. For instance, if profile the same bit of code twice, you may get noticably different runtimes if on one occassion your computational resources were split due to another task taking place on your computer (for instance, if a software update is taking place). This means that a code needs to run for at least a few seconds for profiling to produce meaningful results.

One solution if you're working with a shorter piece of code is to run it many times and look at the total time taken. However, if this is the case, you need to make sure that the overhead of looping over the piece of code to be profiled is not taking up too large a portion of the run-time.

### Profiling Effective - Takeaway

There isn't a simple answer to ensure you get an effective profile of your code as it may be very problem-specific. You should try to think about what your code is doing and what you want to profile carefully and develop a strategy. Then repeat the profiling a few times to ensure the results remain the same.

## Run-time Profiling

Profiling the run-time of the code is probably one of the most important parts of profiling. In many cases, the amount of time the code takes to run is the most limiting factor when deciding if a particular simulation can be carried out on the available computational resources in a reasonable amount of time. There are several useful bits of useful information we can gather.

### Profiling By Function
Profiling by function gives us a high-level idea of how often functions are called and how long those calls last. One way to do this is to import the ```cProfile``` module and run a function using the ```cProfile.run()``` function, providing a string argument which is the command used to invoke the function.

In [1]:
import cProfile

def is_even(value):
  if value%2 == 0:
    return True
  else:
    return False

def halve(value):
  return(value / 2)

def function_to_test(upper_value):
  result = 0
  for i in range(int(upper_value) +1):
    if is_even(i):
      result=result + halve(i)

  print(result)

cProfile.run('function_to_test(1e7)')


12500002500000.0
         15000041 function calls in 3.576 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.135    2.135    3.576    3.576 <ipython-input-1-b2d4b88bdf45>:12(function_to_test)
 10000001    1.018    0.000    1.018    0.000 <ipython-input-1-b2d4b88bdf45>:3(is_even)
  5000001    0.423    0.000    0.423    0.000 <ipython-input-1-b2d4b88bdf45>:9(halve)
        1    0.000    0.000    3.576    3.576 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:382(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:342(send)
        3    0.000    0.000    0.000    0.000 thr

The results from this show how many times each function is called, the total spent within that function (excluding time spent in functions called from this function), the amount of time spent on that function per call (this may be rounded to zero if the run-time is very short), the time spent in that function and other functions called from it and the time spent in that function and other functions called from it per call.

There will normally be a number of functions which are not functions you are written or will be aware of having called. These are often called for reasons internal to how Python is executed and are normally not very consequential in terms of run-time and can often be ignored.

### Call Graphs
Knowing in what context a function is called is very useful for understanding the performance of a code through profiling. This allows you to understand the number of the times different functions call each other and to understand potentially complicated webs of funciton calls and understand why the code is spending the time it does in each function.

Sometimes, this information will be relatively straightforward due to the structure of the code. In more complex codes, it will not always be obvious. In these cases, generating a call graph can be a useful way to obtain and visualise this information.

One example of a Python module capable of doing this is [```pycallgraph```](http://pycallgraph.slowchop.com/en/master/index.html). An example is shown below:

In [2]:
#Install the module using pip
!pip install pycallgraph

#Load the module in the notebook environment
%load_ext pycallgraph

#Import the relevant modules
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

#Define the modules to be tested
def is_multiple_of_four(value):
  if value%2 == 0:
    return True
  else:
    return False

def halve(value):
  return(value / 2)

def quarter(value):
  return(halve(halve(value)))

def function_to_test(upper_value):
  result = 0
  for i in range(int(upper_value) +1):
    if is_multiple_of_four(i):
      result=result + halve(i)+quarter(i)

  print("The result is ", result)

#Call the function the call graph is to be generated for with an instance of the PyCallGraph class
with PyCallGraph(output=GraphvizOutput()):
    function_to_test(1e5)

  Created wheel for pycallgraph: filename=pycallgraph-1.0.1-cp36-none-any.whl size=35943 sha256=98a058e1adc4c4e97d496036b96bda717a9b902a7ead22ad714d50ff981c3cb0
  Stored in directory: /root/.cache/pip/wheels/8c/4b/98/80b30394d77899669da7b8502cbc890d0f35e469022a66eeb3
Successfully built pycallgraph
The pycallgraph module is not an IPython extension.
The result is  1875037500.0


This will have created a file named ```pycallgraph.png```. In Colab, this will be in the local filespace, which can be accessed by opening the left tab (look for the chevron on the left sie of the screen) and selecting "Files". 

The resultant file shows the hierarchy of which functions have called which other functions.

### Profiling By Line

Sometimes, knowing why a code spends a lot of time in a function doesn't provide enough granularity to know exactly which operations are costly. In these cases, there are several options for find out how much time the code spends executing each line. One example is the  [```line_profiler```](https://pypi.org/project/line-profiler/) module.

The first two lines of the following cell install and load this module. The final line uses the line profiler to call the function ```function_to_test``` with a value of $10^6$ passed to the ```upper_value``` argument. The function ```is_even``` is specified as the function to be profiled.

Running the below cell calls ```function_to_test``` and reports the number of times each line is executed, the length of each execuation and the total and fraction of time spent on each line of the function being profiled.

In [4]:
#Install the relevant module
!pip install line_profiler

#Load the module
%load_ext line_profiler

def is_even(value):
  if value%2 == 0:
    return True
  else:
    return False

def halve(value):
  return(value / 2)

def function_to_test(upper_value):
  result = 0
  for i in range(int(upper_value) +1):
    if is_even(i):
      result=result + halve(i)

  print("The result is: ", result)

#"lprun" invokes the line profiler
#We specify "is_even" as the function we want to profile
#"function_to_test(1e6)" specifies the command we want to be executed
#This allows the performance of a function to be tested in a specified context
%lprun -f is_even function_to_test(1e6)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The result is:  125000250000.0


In Colab, the results will be presented in a pop-up labeled "Help" in the bottom right of the screen. This displays, for each line number, the number of times it's called and the time spent on each line in total and per call. It also show the percentage of the total time spent on that line and the contents of the line. Note that all times will need to be multiplied by the "Timer unit" (likely 1$\mu$s) to find the actual time taken.

### Exercise

The code in the cell below has a particular bottleneck that determines the bulk of the runtime of the code.

Examine the code by eye and try to understand how it works, where the bottleneck might lie and what you might do to optimise it. Don't run or change the code at this point.


In [0]:
# The goal of this code is to print either "X is is prime" or "X is not prime" for every value of X between 1 and 1000

import math

# This method checks if "value" is prime and returns True if it is and False if it isn't
def check_prime(value):
  #Initially assume the value is prime
  prime=True

  # Loop over the values which value may be divisible by and check if it's a multiple of any of them
  for i in range(2, int(math.sqrt(value))+1):
    if value%i==0:
      # If value is divisible by i, value%i will be zero, so set prime to false
      prime=False

  # Now we've checked if value is divisible by any of the values it might be divisible by, return prime
  return(prime)

# This function prints a message dependent on whether the given value is prime or not
def print_value_prime(value):
  if check_prime(value):
    print(value, " is prime")
  else:
    print(value, " is not prime")

def print_primes(max_value):
# Loop over all numbers between 1 and max_value, printing if they're prime or not
  for i in range(1, max_value+1):
    print_value_prime(i)

#Print whether values between 1 and 1000 are prime
print_primes(1000)

Use ```cProfile``` and/or ```pycallgraph``` to work out which function this bottleneck lies in, then use ```line_profiler``` to work out which line(s) the bottleneck lies in. Does this match where you thought it might be by eye? Would it be worth performing the optimisations you came up with earlier?

In [7]:
#@title

import math
import cProfile

!pip install line_profiler
%load_ext line_profiler

# This method checks if "value" is prime and returns True if it is and False if it isn't
def check_prime(value):
  #Initially assume the value is prime
  prime=True

  # Loop over the values which value may be divisible by and check if it's a multiple of any of them
  for i in range(2, int(math.sqrt(value))+1):
    if value%i==0:
      # If value is divisible by i, value%i will be zero, so set prime to false
      prime=False

  # Now we've checked if value is divisible by any of the values it might be divisible by, return prime
  return(prime)

# This function prints a message dependent on whether the given value is prime or not
def print_value_prime(value):
  if check_prime(value):
    print(value, " is prime")
  else:
    print(value, " is not prime")

def print_primes(max_value):
# Loop over all numbers between 1 and max_value, printing if they're prime or not
  for i in range(1, max_value+1):
    print_value_prime(i)

#Print whether values between 1 and 1000 are prime
cProfile.run("print_primes(1000)")
#%lprun -f print_value_prime print_primes(1000)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
1  is prime
2  is prime
3  is prime
4  is not prime
5  is prime
6  is not prime
7  is prime
8  is not prime
9  is not prime
10  is not prime
11  is prime
12  is not prime
13  is prime
14  is not prime
15  is not prime
16  is not prime
17  is prime
18  is not prime
19  is prime
20  is not prime
21  is not prime
22  is not prime
23  is prime
24  is not prime
25  is not prime
26  is not prime
27  is not prime
28  is not prime
29  is prime
30  is not prime
31  is prime
32  is not prime
33  is not prime
34  is not prime
35  is not prime
36  is not prime
37  is prime
38  is not prime
39  is not prime
40  is not prime
41  is prime
42  is not prime
43  is prime
44  is not prime
45  is not prime
46  is not prime
47  is prime
48  is not prime
49  is not prime
50  is not prime
51  is not prime
52  is not prime
53  is prime
54  is not prime
55  is not prime
56  is not prime
57  is not prime
58  is not pri

## Memory Profiling



It's also possible to profile the memory which is used by the code. The amount of memory used will change over time as the code runs and data is created and discarded. One meaningful measuerment which can be made is the difference in the amount of memory used at two different points in the program. If the first point is the start of the program then this will give a good approximation of how much memory is in use at the second point. Often it will be desirable for the second point to be when the maximum memory is in use and this may take some experimentation to find.

One example of a memory profiler in Python is [```pympler```](https://pympler.readthedocs.io/en/latest/index.html#). An example is found below:

In [8]:
#Install pympler using pip
!pip install pympler

#Import the relevant parts of pympler
from pympler import summary, muppy

#Get the first measure of the objects in memory
sum1 = muppy.get_objects()

#Create some new objects in memory
list1=[]

for i in range(100):
  new_dict = {}
  for j in range(100):
    new_dict[j] = [1000+i*100+j]
  
  list1.append(new_dict)

#Get the second measure of the objects in memory
sum2 = muppy.get_objects()

#Print a comparison the two measure of the objects in memory
summary.print_(summary.get_diff(summary.summarize(sum1), summary.summarize(sum2)))

     |████████████████████████████████| 184kB 3.2MB/s 
  Created wheel for pympler: filename=Pympler-0.8-cp36-none-any.whl size=164711 sha256=4aba5aa4d9113060048add6927139296b9d6d47b49fe03b5e8a5fa4cd475c837
  Stored in directory: /root/.cache/pip/wheels/6c/61/cc/4bdf1e8c8b1c04d8104322eb2508b6a21d4d3a4d1b44fdd510
Successfully built pympler
                              types |   # objects |   total size
=================================== | =========== | ============
                               list |       10003 |      3.24 MB
                               dict |         101 |    459.48 KB
                                int |       10000 |    273.44 KB
                               code |           1 |    144     B
                              tuple |           1 |     72     B
                             method |           1 |     64     B
                        _ast.Module |           1 |     56     B
             function (is_shadowed) |           0 |      0     B
   functi

The summry printed shows the change in the number of different types of object stored in memory and the change in the amount of memory allocated to each type of object. In the example above, we have created a list containing 100 dictionaries, each containing 100 lists with each list containing one unique integer. Note that there are also a small number of other values created. These may be created due to processes happening "under the hood". Given their small contribution to the total number of values stored, they're not very significant.